## RabattCorner

In [1]:
from fastai.tabular import *

### Dataset

Read dataset

In [2]:
PATH='./'

In [3]:
table_names = ['test', 'train_validation']

In [4]:
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False) for fname in table_names]

In [5]:
test, train = tables

Verify dataset

In [6]:
train.head()

,start_year,start_month,user_id,signup_year,signup_month,postal0,postal1,postal2,postal3,gender,language,birthday,partner_id,previous_visit_count,sum_amount
0,2017,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,7,2,0.0
1,2017,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,12,0,0.0
2,2017,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,16,1,0.0
3,2017,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,19,0,0.0
4,2017,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,20,8,0.0


In [7]:
train.tail()

,start_year,start_month,user_id,signup_year,signup_month,postal0,postal1,postal2,postal3,gender,language,birthday,partner_id,previous_visit_count,sum_amount
50688370,2019,1,44208,2018,11,3.0,8.0,0.0,0.0,1.0,1,1945.0,861,0,0.0
50688371,2019,1,44208,2018,11,3.0,8.0,0.0,0.0,1.0,1,1945.0,862,0,0.0
50688372,2019,1,44208,2018,11,3.0,8.0,0.0,0.0,1.0,1,1945.0,864,0,0.0
50688373,2019,1,44208,2018,11,3.0,8.0,0.0,0.0,1.0,1,1945.0,868,0,0.0
50688374,2019,1,44208,2018,11,3.0,8.0,0.0,0.0,1.0,1,1945.0,869,0,0.0


In [8]:
test.head()

,start_year,start_month,user_id,signup_year,signup_month,postal0,postal1,postal2,postal3,gender,language,birthday,partner_id,previous_visit_count,sum_amount
0,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,7,2,0.0
1,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,12,0,0.0
2,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,16,1,0.0
3,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,19,0,0.0
4,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,20,11,0.0


In [9]:
test.tail()

,start_year,start_month,user_id,signup_year,signup_month,postal0,postal1,postal2,postal3,gender,language,birthday,partner_id,previous_visit_count,sum_amount
2027530,2019,2,44208,2018,11,3.0,8.0,0.0,0.0,1.0,1,1945.0,861,0,0.0
2027531,2019,2,44208,2018,11,3.0,8.0,0.0,0.0,1.0,1,1945.0,862,0,0.0
2027532,2019,2,44208,2018,11,3.0,8.0,0.0,0.0,1.0,1,1945.0,864,0,0.0
2027533,2019,2,44208,2018,11,3.0,8.0,0.0,0.0,1.0,1,1945.0,868,0,0.0
2027534,2019,2,44208,2018,11,3.0,8.0,0.0,0.0,1.0,1,1945.0,869,0,0.0


### Remove NAN values

Next we'll fill in missing values to avoid complications with NA's. NA (not available) is how Pandas indicates missing values; many models have problems when missing values are present, so it's always important to think about how to deal with them. In these cases, we are picking an arbitrary signal value that doesn't otherwise appear in the data.

In [10]:
for df in (test, train):
    df['postal0'] = df.postal0.fillna(-1)
    df['postal1'] = df.postal1.fillna(-1)
    df['postal2'] = df.postal2.fillna(-1)
    df['postal3'] = df.postal3.fillna(-1)
    df['gender'] = df.gender.fillna(0)
    df['birthday'] = df.birthday.fillna(1000)

In [11]:
test.isnull().values.any()

False

In [12]:
train.isnull().values.any()

False

In [13]:
dep_var = 'sum_amount'
cat_names = ['start_year', 'start_month', 'user_id', 'signup_year', 'signup_month', 'postal0', 'postal1', 'postal2', 'postal3', 'gender', 'language', 'birthday', 'partner_id']
cont_names = ['previous_visit_count']
procs = [FillMissing, Categorify, Normalize]

In [14]:
val_idx = list(range(len(train) - len(test), len(train)))

In [15]:
train.iloc[val_idx]

,start_year,start_month,user_id,signup_year,signup_month,postal0,postal1,postal2,postal3,gender,language,birthday,partner_id,previous_visit_count,sum_amount
48660840,2019,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,7,2,0.0
48660841,2019,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,12,0,0.0
48660842,2019,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,16,1,0.0
48660843,2019,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,19,0,0.0
48660844,2019,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,20,11,0.0
48660845,2019,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,26,0,0.0
48660846,2019,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,31,0,0.0
48660847,2019,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,32,0,0.0
48660848,2019,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,34,2,0.0
48660849,2019,1,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,46,0,0.0


In [16]:
test

,start_year,start_month,user_id,signup_year,signup_month,postal0,postal1,postal2,postal3,gender,language,birthday,partner_id,previous_visit_count,sum_amount
0,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,7,2,0.0
1,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,12,0,0.0
2,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,16,1,0.0
3,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,19,0,0.0
4,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,20,11,0.0
5,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,26,0,0.0
6,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,31,0,0.0
7,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,32,0,0.0
8,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,34,2,0.0
9,2019,2,1,2012,12,8.0,0.0,5.0,5.0,1.0,1,1984.0,46,0,0.0


In [17]:
testTL = TabularList.from_df(test.copy(), path=PATH, cat_names=cat_names, cont_names=cont_names)

In [ ]:
dataTL = (TabularList.from_df(train, path=PATH, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(val_idx)
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())